<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-object-detection/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体検出（Object detection）
画像や動画の中から特定の物体を検出し、その位置を特定する技術

<br>

## 物体検出のアーキテクチャ
### 2段階検出器（Two-stage detectors）
まず候補領域を抽出し、次にその候補領域に対して分類と位置の特定を行うという2段階の処理を行う手法。精度は高いが処理速度は遅い。

#### Faster R-CNN
領域提案ネットワーク (Region Proposal Network: RPN)で候補領域を抽出し、Fast R-CNNによって物体の分類、位置の予測を行う。

#### Mask R-CNN
Faster R-CNNの拡張。物体のマスクを予測するマスクブランチをネットワークに追加し、Rolプーリングの代わりにRolAlignを採用している。

<br>

### 1段階検出器（One-stage detectors）
候補領域の抽出と分類・位置の特定を同時に行う手法。処理速度は速いが精度は低い。

#### YOLO（You Only Look Once）
画像全体を一度だけ見て、物体の検出を行う。以下のステップで処理を行う。
1. 画像をグリッドに分割
2. 各グリッドセルでバウンディングボックスとクラス確率を予測
3. Non-Maximum Suppression (NMS) で重複するバウンディングボックスを削除

#### SSD（Single Shot Multibox Detector）
YOLOとは異なるアプローチの物体検出。以下の特徴を持つ。
1. 複数スケールの特徴マップを使用
2. デフォルトボックス (アンカーボックス)を基準にして、物体の位置とクラスを予測
3. 畳み込み層でバウンディングボックスとクラス確率を予測
4. Non-Maximum Suppression (NMS)で最も確度の高いバウンディングボックスを残す

#### RetinaNet
Focal Lossという損失関数を導入して1段階検出器の精度を向上させた。以下の特徴を持つ。
1. Feature Pyramid Network (FPN)で異なる大きさの物体を検出
2. Focal Lossでクラスの不均衡問題の対処を行う
